In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Unequal Weighting
loss_function = IntermediateScaledLoss(omega = 0.1)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-.1/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.034284711153970825
Average test loss: 0.011314661231305864
Epoch 2/300
Average training loss: 0.01264155189279053
Average test loss: 0.010587815262377263
Epoch 3/300
Average training loss: 0.011298432192868657
Average test loss: 0.009881245691743161
Epoch 4/300
Average training loss: 0.01062683865096834
Average test loss: 0.008677920708225833
Epoch 5/300
Average training loss: 0.010071788743966156
Average test loss: 0.008914146616227097
Epoch 6/300
Average training loss: 0.009596516640649902
Average test loss: 0.009041640475806263
Epoch 7/300
Average training loss: 0.009181352467172676
Average test loss: 0.0077993762161996626
Epoch 8/300
Average training loss: 0.008886910840041107
Average test loss: 0.007865392111655739
Epoch 9/300
Average training loss: 0.008644668077429135
Average test loss: 0.007698972972730795
Epoch 10/300
Average training loss: 0.008475303853551547
Average test loss: 0.007613205102582773

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 13.21
Average PSNR for Projection Layer 1 across 2500 images: 19.21
Average PSNR for Projection Layer 2 across 2500 images: 21.52
Average PSNR for Projection Layer 3 across 2500 images: 24.21
Average PSNR for Projection Layer 4 across 2500 images: 25.43
Average PSNR for Projection Layer 0 across 2500 images: 15.93
Average PSNR for Projection Layer 1 across 2500 images: 23.50
Average PSNR for Projection Layer 2 across 2500 images: 25.65
Average PSNR for Projection Layer 3 across 2500 images: 27.23
Average PSNR for Projection Layer 4 across 2500 images: 28.36
Average PSNR for Projection Layer 0 across 2500 images: 17.73
Average PSNR for Projection Layer 1 across 2500 images: 25.05
Average PSNR for Projection Layer 2 across 2500 images: 27.56
Average PSNR for Projection Layer 3 across 2500 images: 29.27
Average PSNR for Projection Layer 4 across 2500 images: 30.43
Average PSNR for Projection Layer 0 across 2500 images: 18.94
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-.1/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.08921172893047333
Average test loss: 0.05556458235449261
Epoch 2/300
Average training loss: 0.015404069806138674
Average test loss: 0.015731954554716748
Epoch 3/300
Average training loss: 0.013579661950469018
Average test loss: 0.01043929080499543
Epoch 4/300
Average training loss: 0.012342524657646814
Average test loss: 0.01021995196160343
Epoch 5/300
Average training loss: 0.011452551224165492
Average test loss: 0.018809722706675528
Epoch 6/300
Average training loss: 0.010612828117277887
Average test loss: 0.03060338205595811
Epoch 7/300
Average training loss: 0.009983162984665898
Average test loss: 0.00839532863183154
Epoch 8/300
Average training loss: 0.009515324172874292
Average test loss: 0.007871354819171958
Epoch 9/300
Average training loss: 0.00912248805248075
Average test loss: 0.007813014616154962
Epoch 10/300
Average training loss: 0.008697217533157931
Average test loss: 0.00726207470231586
Epoch 

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 5.34
Average PSNR for Projection Layer 1 across 2500 images: 11.50
Average PSNR for Projection Layer 2 across 2500 images: 16.09
Average PSNR for Projection Layer 3 across 2500 images: 14.88
Average PSNR for Projection Layer 4 across 2500 images: 20.58
Average PSNR for Projection Layer 5 across 2500 images: 21.32
Average PSNR for Projection Layer 6 across 2500 images: 10.49
Average PSNR for Projection Layer 7 across 2500 images: 20.80
Average PSNR for Projection Layer 8 across 2500 images: 22.74
Average PSNR for Projection Layer 9 across 2500 images: 24.69
Average PSNR for Projection Layer 10 across 2500 images: 15.03
Average PSNR for Projection Layer 11 across 2500 images: 24.03
Average PSNR for Projection Layer 12 across 2500 images: 23.97
Average PSNR for Projection Layer 13 across 2500 images: 26.32
Average PSNR for Projection Layer 14 across 2500 images: 26.73
Average PSNR for Projection Layer 0 across 2500 images: 5.87
Avera

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-.1/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.26050928941534623
Average test loss: 0.018052405796945094
Epoch 2/300
Average training loss: 0.021455550061331857
Average test loss: 0.014757054846319888
Epoch 3/300
Average training loss: 0.01680204262998369
Average test loss: 0.012095896686116854
Epoch 4/300
Average training loss: 0.014942899543378088
Average test loss: 0.018189512178301812
Epoch 5/300
Average training loss: 0.01397633307509952
Average test loss: 0.011349175732168886
Epoch 6/300
Average training loss: 0.013019312486880355
Average test loss: 0.010014628834194607
Epoch 7/300
Average training loss: 0.01198061959611045
Average test loss: 0.034797214090824126
Epoch 8/300
Average training loss: 0.011420264722572432
Average test loss: 0.020099861840407053
Epoch 9/300
Average training loss: 0.011000640263987912
Average test loss: 0.01058987069543865
Epoch 10/300
Average training loss: 0.011042490445905261
Average test loss: 0.00911638157442212
Epoc

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: -3.19
Average PSNR for Projection Layer 1 across 2500 images: -2.31
Average PSNR for Projection Layer 2 across 2500 images: -4.92
Average PSNR for Projection Layer 3 across 2500 images: -8.59
Average PSNR for Projection Layer 4 across 2500 images: -7.45
Average PSNR for Projection Layer 5 across 2500 images: -7.89
Average PSNR for Projection Layer 6 across 2500 images: -7.92
Average PSNR for Projection Layer 7 across 2500 images: -6.40
Average PSNR for Projection Layer 8 across 2500 images: -8.36
Average PSNR for Projection Layer 9 across 2500 images: -7.15
Average PSNR for Projection Layer 10 across 2500 images: -6.25
Average PSNR for Projection Layer 11 across 2500 images: -5.08
Average PSNR for Projection Layer 12 across 2500 images: -4.35
Average PSNR for Projection Layer 13 across 2500 images: -2.05
Average PSNR for Projection Layer 14 across 2500 images: 1.03
Average PSNR for Projection Layer 15 across 2500 images: 3.36
Aver

In [13]:
import shutil

def zip_folder(source_folder, output_zip_file):
    # Ensure the output does not contain an extension (shutil will append .zip)
    output_zip_file = output_zip_file.rstrip('.zip')

    # Zip the folder
    shutil.make_archive(output_zip_file, 'zip', source_folder)

# Define the folder to zip and the location of the output file
source_folder = 'Memory_Residual-.1'  # Replace with the path to your folder
output_zip_file = 'Memory_Residual-.1.zip'  # Replace with the new zip location without .zip

# Call the function to zip the folder
zip_folder(source_folder, output_zip_file)
